In [2]:
import gradio as gr
from transformers import pipeline
from PIL import Image
import requests
from io import BytesIO

In [3]:
# Load captioning model
captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")

Device set to use cuda:0


In [8]:
def caption_from_upload(image):
    if image is None:
        return None, "Please upload an image."
    result = captioner(image)
    return image, result[0]['generated_text']

def caption_from_url(image_url):
    try:
        if not image_url:
            return None, "Please enter an image URL."
        response = requests.get(image_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert("RGB")
        result = captioner(image)
        return image, result[0]['generated_text']
    except Exception as e:
        return None, f"Error: {str(e)}"

In [11]:
with gr.Blocks() as demo:
    gr.Markdown("## Image Captioning App")

    with gr.Tabs():
        with gr.Tab("Upload Image"):
            with gr.Row():
                uploaded_image = gr.Image(type="pil", label="Upload an Image")
            with gr.Row():
                output_image1 = gr.Image(label="Processed Image")
                output_caption1 = gr.Textbox(label="Generated Caption")
            generate_btn1 = gr.Button("Generate Caption")
            generate_btn1.click(fn=caption_from_upload, inputs=uploaded_image, outputs=[output_image1, output_caption1])

        with gr.Tab("Use Image URL"):
            with gr.Row():
                image_url_input = gr.Textbox(label="Enter Image URL")
            with gr.Row():
                output_image2 = gr.Image(label="Fetched Image")
                output_caption2 = gr.Textbox(label="Generated Caption")
            generate_btn2 = gr.Button("Generate Caption")
            generate_btn2.click(fn=caption_from_url, inputs=image_url_input, outputs=[output_image2, output_caption2])

In [12]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://0a1a089a9f64b99a1e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
